In [1]:
import pandas as pd
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 170)
pd.set_option('display.max_colwidth', 100000)
import numpy as np
from matplotlib import pyplot as plt
import torch
import fastai
from fastai.vision.all import *
from fastai.distributed import *
import sklearn.metrics as skm
from unidecode import unidecode


Bad key "text.kerning_factor" on line 4 in
/home/auri/anaconda3/envs/nlpMED/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
path = Path('clinical_scales')
path.ls()

(#4) [Path('clinical_scales/charlson.csv'),Path('clinical_scales/apacheII.csv'),Path('clinical_scales/non_ICD.csv'),Path('clinical_scales/ICD_covid.csv')]

In [3]:
#load charlson icd table
apache_icd = pd.read_csv( path /'apacheII.csv', delimiter = '\t')
apache_icd

,ICD10,desc,category
0,I25.5,"Cardiomyopathy, ischemic",Cardiovascular
1,I42.0,"Cardiomyopathy, dilated",Cardiovascular
2,I42.1,"Cardiomyopathy, hypertrophic obstructive (HOCM, IHSS)",Cardiovascular
3,I42.6,"Cardiomyopathy, alcoholic",Cardiovascular
4,I42.7,"Cardiomyopathy, due to infection, drugs or external agents NOS",Cardiovascular
...,...,...,...
100,J84.89,"Bronchiolitis obliterans organizing pneumonia (BOOP, cryptogenic organizing pneumonia (COP))",Respiratory
101,J84.9,"Interstitial lung disease, NOS",Respiratory
102,J96.19,"Respiratory failure (insufficiency), chronic",Respiratory
103,J96.99,"Respiratory failure (insufficiency) NOS, not specified as acute or chronic",Respiratory


In [4]:

#load charlson icd table
charl_icd = pd.read_csv( path /'charlson.csv')
charl_icd.ICD10 = charl_icd.ICD10.str.replace('\s','', regex = True).str.split(';')
charl_icd

,comorbidity_category,condition,ICD8,ICD10,CCIScore,substract_condition
0,Cardiopulmonary disease,Myocardial infarction,410,"[I21, I22, I23]",1,NaN
1,Cardiopulmonary disease,Congestive heart failure,427.09;427.10; 427.11; 427.19; 428.99; 782.49,"[I50, I11.0, I13.0, I13.2]",1,NaN
2,Cardiopulmonary disease,Chronic pulmonary disease,490;491;492;493; 515;516;517;518,"[J40, J41, J42, J43, J44, J45, J46, J47, J60, J61, J62, J63, J64, J65, J66, J67, J68.4, J70.1, J70.3, J84.1, J92.0, J96.1, J98.2, J98.3]",1,NaN
3,Gastrointestinal disease,Ulcer disease,530.91; 530.98; 531–534,"[K22.1, K25, K26, K27, K28]",1,NaN
4,Gastrointestinal disease,Mild liver disease,571; 573.01; 573.04,"[B18, K70.0, K70.1, K70.2, K70.3, K70.9, K71, K73, K74, K76.0]",1,NaN
5,Gastrointestinal disease,Moderate/severe liver disease,070.00; 070.02; 070.04; 070.06;070.08; 456.00–456.09; 573.00,"[B15.0, B16.0, B16.2, B19.0, K70.4, K72, K76.6, I85]",3,Mild liver disease
6,Neurovascular disease,Peripheral vascular disease,440–445,"[I70, I71, I72, I73, I74, I77]",1,NaN
7,Neurovascular disease,Cerebrovascular disease,430–438,"[I60, I61, I62, I63, I64, I65, I66, I67, I68, I69, G45, G46]",1,NaN
8,Neurovascular disease,Dementia,290.09–290.19; 293.09,"[F00, F01, F02, F03, F05.1, G30]",1,NaN
9,Neurovascular disease,Hemiplegia,344,"[G81, G82]",2,NaN


In [5]:
def substract_conditions(): #substract overlapping criteria, e.g. if DM with end organ damage, then DM should not be scored twice
    df = charl_icd[(charl_icd.substract_condition.isna() == False) & (charl_icd.check > 0)]
    for i,r in df.iterrows():
        charl_icd.loc[(charl_icd.condition == r.substract_condition),['check']] = 0


In [6]:
minmax_norm_creat = 1
icd_list = ['C00', 'C01', 'C02', 'C03','E11.61', 'B22',  'E11.9']
age = 1
def charlson(icd_list = None, minmax_norm_creat = None, age = None):
    charl_icd['check'] = 0
    #apply age criteria
    score = ((age - 50) // 10) + 1 if age >= 50 else 0 
    
    #apply icd criteria
    charl_icd['check'] = charl_icd.apply(lambda z : z.CCIScore if any( x.startswith(tuple(z.ICD10)) for x in  icd_list) else 0, axis = 1)
    substract_conditions() 
    
    #apply lab criteria
    ##creatinine >3 mg/dL (0.27 mmol/L): if > min-max norm value then Moderate/severe renal disease = True
    if charl_icd.loc[charl_icd.condition.str.contains('Moderate/severe renal disease'), ['check']].values[0] == 0:
        creat_range = [0.84, 1.21] #range 0.84 to 1.21 mg/dl
        minmax_norm_creat_limit = (3 - creat_range[0])/(creat_range[1]- creat_range[0])
        charl_icd.loc[charl_icd.condition.str.contains('Moderate/severe renal disease'), ['check']] = charl_icd.CCIScore if (minmax_norm_creat > minmax_norm_creat_limit) else charl_icd.check

    score +=  charl_icd.check.sum()
    
    return score

charlson(icd_list,minmax_norm_creat, age)

10

In [ ]:
charl_icd